In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np
import json
import seaborn as sns
import warnings
from urllib.request import urlopen
from plotly import graph_objects as go
from sklearn import datasets
from sklearn.linear_model import LinearRegression
from dotenv import load_dotenv




In [ ]:
load_dotenv()

In [3]:
Sub_National_Energy_Consumption_1 = os.getenv('Sub_National_Energy_Consumption_1')
Sub_National_Gas_Consumption_2 = os.getenv('Sub_National_Gas_Consumption_2')
UK_Population_3 = os.getenv('UK_Population_3') 
Local_Authority_District_to_Region_4 = os.getenv('Local_Authority_District_to_Region_4') 
Domestic_Energy_Bills_5 = os.getenv('Domestic_Energy_Bills_5') 
Energy_Import_Costs_6 = os.getenv('Energy_Import_Costs_6') 
Average_Earning_7 = os.getenv('Average_Earning_7')

In [4]:
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
#Defining the sheet names to read from the excel
Years = [str(n) for n in list(range(2005,2023))]

#Defining the dataframe which will hold the Energy consumption data as the concatination of sheets in the excel with the Year appended
Energy_Consumption = pd.concat(((pd.read_excel(io=Sub_National_Energy_Consumption_1, sheet_name=Year, header=5, nrows=11, skiprows=[6,7,8,9,10,18,19]).assign(Year = Year))for Year in Years), ignore_index=True)
Energy_Consumption['Country or region'].unique()

In [ ]:
#Getting the sum of energy used in England for each year
Total_Energy_Consumption = Energy_Consumption.groupby('Year').sum()

#Rename the column for improved readability
Total_Energy_Consumption.rename(columns= {'All fuels:\nTotal': 'Total Energy Consumption (ktoe)'},inplace=True)
Total_Energy_Consumption['Total Energy Consumption (GWh)'] = Total_Energy_Consumption['Total Energy Consumption (ktoe)'] * 11.63
Total_Energy_Consumption[['Total Energy Consumption (GWh)','Total Energy Consumption (ktoe)']]

In [7]:
#For this datasource not all the sheets have the same format, so the read_excel function must be adjusted for each.

#Defining the sheet names to read from the excel
Years1 = [str(n) for n in list(range(2005,2014))]
Gas_Consumption1 = pd.concat(((pd.read_excel(io=Sub_National_Gas_Consumption_2, sheet_name=Year, header=5, nrows=9, skiprows=[6,7]).assign(Year = Year))for Year in Years1), ignore_index=True)

#In this one Inner and outerlondon are differinciated
Years2 = [str(n) for n in list(range(2014,2015))]
Gas_Consumption2 = pd.concat(((pd.read_excel(io=Sub_National_Gas_Consumption_2, sheet_name=Year, header=5, nrows=10, skiprows=[6,7]).assign(Year = Year))for Year in Years2), ignore_index=True)

#Removing the sub regions of Inner and outer London as as sum of these 'London is already included' this is to avoid double counting
Years3 = [str(n) for n in list(range(2015,2023))]
Gas_Consumption3 = pd.concat(((pd.read_excel(io=Sub_National_Gas_Consumption_2, sheet_name=Year, header=5, nrows=11, skiprows=[6,7,8,16,17]).assign(Year = Year))for Year in Years3), ignore_index=True)


#Combined data from all sheets into one dataframe
Gas_Consumption = pd.concat([Gas_Consumption1, Gas_Consumption2, Gas_Consumption3])


In [ ]:
#Getting the sum of gas used in England for each year
Total_Gas_Consumption = Gas_Consumption.groupby('Year').sum()

#Rename the column for improved readability
Total_Gas_Consumption.rename(columns= {'Total consumption\n(GWh):\nAll meters': 'Total Gas Consumption (GWh)'},inplace=True)
Total_Gas_Consumption['Total Gas Consumption (GWh)'] 

In [ ]:
#Joining the Energy and Gas consumption by Year dataframes together
Energy_Gas_Consumption_Combined = pd.merge(Total_Energy_Consumption,Total_Gas_Consumption, how ='left', on=['Year','Year'])

#Calculating the percentage of total energy consumption gas consumption makes up
Energy_Gas_Consumption_Combined['Gas_Consumption_as_Percent'] = Energy_Gas_Consumption_Combined['Total Gas Consumption (GWh)']/Energy_Gas_Consumption_Combined['Total Energy Consumption (GWh)']*100
Energy_Gas_Consumption_Combined['Other_Consumption_as_Percent'] = 100 - Energy_Gas_Consumption_Combined['Gas_Consumption_as_Percent']
#fixing the year index
Energy_Gas_Consumption_Combined = Energy_Gas_Consumption_Combined.reset_index(names=['Year_x','Year'])
Energy_Gas_Consumption_Combined = Energy_Gas_Consumption_Combined.drop(columns='Year_x', axis=0)

#Printing the dataframe columns which will be plotted
Energy_Gas_Consumption_Combined[['Year', 'Total Energy Consumption (GWh)','Total Gas Consumption (GWh)','Gas_Consumption_as_Percent','Other_Consumption_as_Percent']]


In [ ]:
Energy_Consumption_Fig = px.line(Energy_Gas_Consumption_Combined, x='Year', y=['Total Energy Consumption (GWh)','Total Gas Consumption (GWh)'])

Energy_Consumption_Fig.update_layout(
    title="Energy Consumption in the UK by year (GWh)",
    xaxis_title="Year",
    yaxis_title="Energy Consumption (GWh)",
    title_x=0.5, 
    legend=dict(
        orientation="h", 
        yanchor="bottom", 
        y=0.88, 
        xanchor="center",  
        x=0.5,
        title = None
    ),
    height = 700,
    font=dict(
        size=26 
    )
)

Energy_Consumption_Fig

In [ ]:
Gas_Consumption_as_Percentage_for_Chart = Energy_Gas_Consumption_Combined[['Year','Gas_Consumption_as_Percent','Other_Consumption_as_Percent']]
Gas_Consumption_as_Percentage_for_Chart = Gas_Consumption_as_Percentage_for_Chart.set_index('Year')
#Gas_Consumption_as_Percentage_for_Chart
#Gas_Proportion_Fig = Gas_Consumption_as_Percentage_for_Chart.plot(kind = 'bar', stacked = True )
Gas_Consumption_as_Percentage_for_Chart_Bar = px.bar(Gas_Consumption_as_Percentage_for_Chart)
Gas_Consumption_as_Percentage_for_Chart_Bar.update_layout(
    margin={"r": 10, "t": 140, "l": 10, "b": 10},
    title='Gas as a Portion of Total Energy Consumption',
    xaxis_title="Year",
    yaxis_title="Percentage Consumption (%)",
    title_x=0.5,
    legend=dict(
        orientation="h", 
        yanchor="bottom", 
        y=1, 
        xanchor="center",  
        x=0.5,
        title = None
    ),
    height = 700,
    font=dict(
        size=26 
    ))



The Importation of Population Data

In [ ]:
#The Importation of Population Data

#Source for this data it's the 'Mid-2011 to mid-2022 edition of this dataset'
#https://www.ons.gov.uk/peoplepopulationandcommunity/populationandmigration/populationestimates/datasets/populationestimatesforukenglandandwalesscotlandandnorthernireland

UK_Population_Raw_Data = pd.read_excel(io=UK_Population_3,sheet_name='MYEB1', header=1)
UK_Population_Raw_Data

In [ ]:
UK_Population_Raw_Data[UK_Population_Raw_Data['laname23']== '']

In [ ]:
#Source for Lookup table:
#https://geoportal.statistics.gov.uk/datasets/ons::local-authority-district-to-region-april-2020-lookup-in-en/explore
#In excel Find and replace was used to change all instances of 'East of England' to 'East'
LAD20_to_RGN20_Conversion_Table = pd.read_excel(io=Local_Authority_District_to_Region_4)
LAD20_to_RGN20_Conversion_Table

In [ ]:
LAD20_to_RGN20_Conversion_Table['RGN20NM'].unique()


In [ ]:
#Join the RGN20NM onto the UK_Population_Density_Raw_Data table
UK_Population_Raw_Data_With_Region = pd.merge(UK_Population_Raw_Data, LAD20_to_RGN20_Conversion_Table, left_on='ladcode23', right_on='LAD20CD', how='left')

#This checks if all rows relating to england have had a RGN20NM successfully allocated
len(UK_Population_Raw_Data_With_Region[UK_Population_Raw_Data_With_Region['RGN20NM'].notna()]) - len(UK_Population_Raw_Data_With_Region[UK_Population_Raw_Data_With_Region['country'] == 'E'])

#This initially returned a difference of 1092
#After manual adjustments this turned to 0

In [ ]:
Region_Allocation_Erros = UK_Population_Raw_Data_With_Region[
    (UK_Population_Raw_Data_With_Region['RGN20NM'].isna()) &
    (UK_Population_Raw_Data_With_Region['country'] == 'E')]

Region_Allocation_Erros['ladcode23'].unique()

#The output here initially was: 'array(['E06000061', 'E06000062', 'E06000063', 'E06000064', 'E06000065', 'E06000066'], dtype=object)'
#These values where allocated manually in the LAD20_to_RGN20_Conversion_Table excel. The the join was performed again.


In [ ]:
#Group by Region
Population_by_Region_Pivoted = UK_Population_Raw_Data_With_Region.groupby('RGN20NM',as_index=False).agg({
    'population_2011' : 'sum',
    'population_2012' : 'sum',
    'population_2013' : 'sum',
    'population_2014' : 'sum',
    'population_2015' : 'sum',
    'population_2016' : 'sum',
    'population_2017' : 'sum',
    'population_2018' : 'sum',
    'population_2019' : 'sum',
    'population_2020' : 'sum',
    'population_2021' : 'sum',
    'population_2022' : 'sum',
})

Population_by_Region_Pivoted


In [ ]:
#Unpivot the year columns

Population_by_Region_and_Year = pd.melt(Population_by_Region_Pivoted, id_vars='RGN20NM')
Population_by_Region_and_Year = Population_by_Region_and_Year.rename(columns={'RGN20NM':'Region','variable':'Year','value':'Population'})
Population_by_Region_and_Year['Year'] = Population_by_Region_and_Year['Year'].apply(lambda x: x.split('_')[1])

Population_by_Region_and_Year

In [ ]:
Population_by_Region_and_Year_Line = px.line(Population_by_Region_and_Year, x= 'Year', y='Population', color='Region')

Population_by_Region_and_Year_Line
Population_by_Region_and_Year_Line.update_layout(
        margin={"r": 10, "t": 170, "l": 10, "b": 10},
    title="Population by Region and Year",
    xaxis_title="Year",
    yaxis_title="Population",
    title_x=0.5,
    title_y=0.95, 
    legend=dict(
        orientation="h", 
        yanchor="bottom", 
        y=1, 
        xanchor="center",  
        x=0.5,
        title = None
    ,font=dict(
        size=26 
    )),
    height = 1000,
    font=dict(
        size=26 
    ))
    
Population_by_Region_and_Year_Line


In [ ]:
Population_by_Region_and_Year_Pivot = Population_by_Region_and_Year.pivot(index = 'Year', columns= 'Region')
Population_by_Region_and_Year_Pivot['Year'] = Population_by_Region_and_Year_Pivot.index
Population_by_Region_and_Year_Pivot.corr()

#High corrolation with Year so exterapolation is viable

In [22]:
def extrapolate_population_for_region_v1(region):
    #Filtering to handle each region seperately
    Population_by_Region_and_Year_for_Extrapoloation = Population_by_Region_and_Year[Population_by_Region_and_Year['Region']==region]

    X = Population_by_Region_and_Year_for_Extrapoloation['Year'].values.reshape(-1,1)
    y = Population_by_Region_and_Year_for_Extrapoloation['Population']

    #Fitting the linear regession model
    Population_by_Region_and_Year_Model = LinearRegression()
    Population_by_Region_and_Year_Model.fit(X,y)
    
    #Extrapolating for the years where data is missing
    years_to_extrapolate = np.array([2005, 2006, 2007, 2008, 2009, 2010]).reshape(-1, 1)

    population_predictions = Population_by_Region_and_Year_Model.predict(years_to_extrapolate)
    population_predictions = population_predictions.astype(int)
    
    #Bring the predictions into a dataframe
    Predictions_df = pd.DataFrame({'Year':years_to_extrapolate.flatten(),'Population':Population_by_Region_and_Year_Model.predict(years_to_extrapolate),'Region':region,'Data_Type': 'Extrapolated'})
    
    return Predictions_df

In [23]:
Population_by_Region_and_Year['Data_Type'] = "Original"

In [24]:
#Create a list of regions for which to loop the extrapolate function
Regions = list(Population_by_Region_and_Year['Region'].unique())

#Concatating all the extrapolated values into one dataframe
Population_by_Region_and_Year_Extrapolated = pd.concat((extrapolate_population_for_region_v1(region) for region in Regions), ignore_index=True)

#Combining the Original data with the extrapolated data
Population_by_Region_and_Year_Combined = pd.concat([Population_by_Region_and_Year,Population_by_Region_and_Year_Extrapolated], ignore_index=True)


In [25]:
# Change the datatype of the Year column to be integer (required step to fix later graph)
Population_by_Region_and_Year_Combined['Year'] = Population_by_Region_and_Year_Combined['Year'].astype(int)

Population_by_Region_and_Year_Combined = Population_by_Region_and_Year_Combined.sort_values(by='Year')

In [ ]:
Population_by_Region_and_Year_Combined[Population_by_Region_and_Year_Combined['Region'] =="Yorkshire and The Humber"]

In [ ]:
Population_by_Region_and_Year_Combined_Line = px.line(Population_by_Region_and_Year_Combined, x= 'Year', y='Population', color='Region')

Population_by_Region_and_Year_Combined_Line
Population_by_Region_and_Year_Combined_Line.update_layout(
        margin={"r": 10, "t": 170, "l": 10, "b": 10},
    title="Population by Region and Year (Extrapolation Model 1)",
    xaxis_title="Year",
    yaxis_title="Population",
    title_x=0.5,
    title_y=0.95, 
    legend=dict(
        orientation="h", 
        yanchor="bottom", 
        y=1, 
        xanchor="center",  
        x=0.5,
        title = None
    ,font=dict(
        size=26 
    )),
    height = 1000,
    font=dict(
        size=26 
    ))

#Include commentry about the 2010 prediction for London this seems unlikely

In [28]:
#Lets check the R^2 value for this way of modeling

In [29]:
def extrapolate_population_for_region_r2_model1(region):
    #Filtering to handle each region seperately
    Population_by_Region_and_Year_Region = Population_by_Region_and_Year[Population_by_Region_and_Year['Region']==region]

    X = Population_by_Region_and_Year_Region['Year'].values.reshape(-1,1)
    y = Population_by_Region_and_Year_Region['Population']

    #Fitting the linear regession model
    Population_by_Region_and_Year_Model = LinearRegression()
    Population_by_Region_and_Year_Model.fit(X,y)
    
    R2 = Population_by_Region_and_Year_Model.score(X, y)
    
    return R2

In [30]:
def extrapolate_population_for_region_r2_model2(region):
    #Filtering to handle each region seperately
    Population_by_Region_and_Year_Region = Population_by_Region_and_Year[(Population_by_Region_and_Year['Region']==region) &
                                                                            (Population_by_Region_and_Year['Year']!='2020') & 
                                                                            (Population_by_Region_and_Year['Year']!='2021') &
                                                                            (Population_by_Region_and_Year['Year']!='2022')]
    
    X = Population_by_Region_and_Year_Region['Year'].values.reshape(-1,1)
    y = Population_by_Region_and_Year_Region['Population']

    #Fitting the linear regession model
    Population_by_Region_and_Year_Model = LinearRegression()
    Population_by_Region_and_Year_Model.fit(X,y)
    
    R2 = Population_by_Region_and_Year_Model.score(X, y)
    
    return R2

In [31]:
R2_outcomes = pd.DataFrame(columns=['Region','Model1_R2','Model2_R2'])

In [ ]:
for region in Regions:
    new_row_for_r2 = pd.DataFrame({'Region': [region],'Model1_R2':[extrapolate_population_for_region_r2_model1(region)],'Model2_R2':[extrapolate_population_for_region_r2_model2(region)]})
    R2_outcomes = pd.concat([R2_outcomes, new_row_for_r2], ignore_index=True)
    
R2_outcomes

In [ ]:
R2_outcomes['Percentage_Improved'] = (R2_outcomes['Model2_R2'] / R2_outcomes['Model1_R2'] - 1)*100
R2_outcomes

In [34]:
def extrapolate_population_for_region_v2(region):
    #Filtering to handle each region seperately
    Population_by_Region_and_Year_for_Extrapoloation2 = Population_by_Region_and_Year[(Population_by_Region_and_Year['Region']==region) &
                                                                            (Population_by_Region_and_Year['Year']!='2020') & 
                                                                            (Population_by_Region_and_Year['Year']!='2021') &
                                                                            (Population_by_Region_and_Year['Year']!='2022')]


    X = Population_by_Region_and_Year_for_Extrapoloation2['Year'].values.reshape(-1,1)
    y = Population_by_Region_and_Year_for_Extrapoloation2['Population']

    #Fitting the linear regession model
    Population_by_Region_and_Year_Model_v2 = LinearRegression()
    Population_by_Region_and_Year_Model_v2.fit(X,y)
    
    #Extrapolating for the years where data is missing
    years_to_extrapolate = np.array([2005, 2006, 2007, 2008, 2009, 2010]).reshape(-1, 1)

    population_predictions = Population_by_Region_and_Year_Model_v2.predict(years_to_extrapolate)
    population_predictions = population_predictions.astype(int)
    
    #Bring the predictions into a dataframe
    Predictions_df = pd.DataFrame({'Year':years_to_extrapolate.flatten(),'Population':Population_by_Region_and_Year_Model_v2.predict(years_to_extrapolate),'Region':region,'Data_Type': 'Extrapolated'})
    
    return Predictions_df

In [35]:
#Create a list of regions for which to loop the extrapolate function
Regions = list(Population_by_Region_and_Year['Region'].unique())

#Concatating all the extrapolated values into one dataframe
Population_by_Region_and_Year_Extrapolated_v2 = pd.concat((extrapolate_population_for_region_v2(region) for region in Regions), ignore_index=True)

#Combining the Original data with the extrapolated data
Population_by_Region_and_Year_Combined_v2 = pd.concat([Population_by_Region_and_Year,Population_by_Region_and_Year_Extrapolated_v2], ignore_index=True)


In [36]:
# Change the datatype of the Year column to be integer (required step to fix later graph)
Population_by_Region_and_Year_Combined_v2['Year'] = Population_by_Region_and_Year_Combined_v2['Year'].astype(int)

Population_by_Region_and_Year_Combined_v2 = Population_by_Region_and_Year_Combined_v2.sort_values(by='Year')

In [ ]:
Population_by_Region_and_Year_Combined_v2_Line = px.line(Population_by_Region_and_Year_Combined_v2, x= 'Year', y='Population', color='Region')

Population_by_Region_and_Year_Combined_v2_Line
Population_by_Region_and_Year_Combined_v2_Line.update_layout(
        margin={"r": 10, "t": 170, "l": 10, "b": 10},
    title="Population by Region and Year (Extrapolation Model 2)",
    xaxis_title="Year",
    yaxis_title="Population",
    title_x=0.5,
    title_y=0.95, 
    legend=dict(
        orientation="h", 
        yanchor="bottom", 
        y=1, 
        xanchor="center",  
        x=0.5,
        title = None
    ,font=dict(
        size=26 
    )),
    height = 1000,
    font=dict(
        size=26 
    ))
#Include commentry about the 2010 prediction for London this seems unlikely

In [ ]:
Population_by_Region_and_Year_Combined_v2[Population_by_Region_and_Year_Combined_v2['Region']=='London']

In [39]:
#Data taken from here:
#https://en.wikipedia.org/wiki/Regions_of_England See citation [28]

Regional_Areas = pd.DataFrame({'Region':['North East','North West','Yorkshire and The Humber', 'East Midlands', 'West Midlands', 'East', 'London', 'South East', 'South West'],'Area km^2':['8581','14108','15404','15624','12998','19116','1572','1972','23836']})

Regional_Areas['Area km^2'] = Regional_Areas['Area km^2'].astype(float)

In [ ]:
Population_Density_by_Region = Population_by_Region_and_Year_Combined_v2.merge(Regional_Areas, on='Region', how='left')
Population_Density_by_Region['Population_Density(Persons.km^-2)'] = Population_Density_by_Region['Population'] / Population_Density_by_Region['Area km^2']
Population_Density_by_Region

Making Chloropleths

Cite this user:
https://github.com/martinjc/UK-GeoJSON/tree/master
Cite this GeoJSON Specifically:
https://raw.githubusercontent.com/martinjc/UK-GeoJSON/refs/heads/master/json/eurostat/ew/nuts1.json

In [41]:
#Importing the GeoJSON
with urlopen('https://raw.githubusercontent.com/martinjc/UK-GeoJSON/refs/heads/master/json/eurostat/ew/nuts1.json') as response:
    Regions = json.load(response)
    

In [ ]:
#Printing a list of all Regions in the GeoJSON along with their Location Code
for ref in range(len(Regions['features'])):
    print(f"{Regions['features'][ref]['properties']['NUTS112CD']} / {Regions['features'][ref]['properties']['NUTS112NM']}")

In [ ]:
#Printing a list of all the Location Codes in the Gas_Consumption dataframe
Gas_Consumption['Country or region'].unique()

In [ ]:
#Defining the Location_Code mapping table
Location_Code_List = [
    ('UKC', 'North East', 'North East'),
    ('UKD', 'North West', 'North West'),
    ('UKE', 'Yorkshire and The Humber', 'Yorkshire and The Humber'),
    ('UKE', 'Yorkshire and the Humber', 'Yorkshire and The Humber'),
    ('UKF', 'East Midlands', 'East Midlands'),
    ('UKG', 'West Midlands', 'West Midlands'),
    ('UKH', 'East', 'East'),
    ('UKI', 'Inner London', 'London'),
    ('UKI', 'Outer London', 'London'),
    ('UKI', 'London', 'London'),
    ('UKJ', 'South East', 'South East'),
    ('UKK', 'South West', 'South West')
]

Location_Code = pd.DataFrame(Location_Code_List, columns=['Location_Code', 'In_Region', 'Region'])
Location_Code

In [ ]:
#This step reclassifies regions into the desired regions found in the Geo-JSON
Gas_Consumption_Cleaned = Gas_Consumption.merge(Location_Code, right_on='In_Region', left_on='Country or region', how='left')

#This step combines values for Outer and inner london, which have both been classified as London
Gas_Consumption_Cleaned = Gas_Consumption_Cleaned.groupby(['Year','Location_Code'],as_index=False).agg({
    'Region': 'max',
    'Number of meters\n(thousands):\nDomestic\n' : 'sum',
     'Number of meters\n(thousands):\nNon-Domestic': 'sum',
     'Number of meters\n(thousands):\nAll meters': 'sum',
     'Total consumption\n(GWh):\nDomestic\n': 'sum',
     'Total consumption\n(GWh):\nNon-Domestic': 'sum',
     'Total consumption\n(GWh):\nAll meters': 'sum',
     'Mean consumption\n(kWh per meter):\nDomestic\n': 'sum',
     'Mean consumption\n(kWh per meter):\nNon-Domestic': 'sum',
     'Mean consumption\n(kWh per meter):\nAll meters' : 'sum'
})
Gas_Consumption_Cleaned

In [ ]:
#Checking that all regions have been successfully allocated a Location_Code
Gas_Consumption_Cleaned[Gas_Consumption_Cleaned['Location_Code'].isnull()]

In [47]:
#Converts the field 'Year' to an integer to allow following join
Gas_Consumption_Cleaned['Year'] = Gas_Consumption_Cleaned['Year'].astype(int)

In [ ]:
#Here I am merging the Population density data onto the Gas energy consumption data
Gas_Consumption_Cleaned_PopD = Gas_Consumption_Cleaned.merge(Population_Density_by_Region, on=['Region','Year'], how='left')

Gas_Consumption_Cleaned_PopD[Gas_Consumption_Cleaned_PopD['Year'] == 2010]

In [ ]:
#Calculated Gas consumption per person for each region and year
Gas_Consumption_Cleaned_PopD['Domestic_Gas_Consumption(GWh/person)'] = Gas_Consumption_Cleaned_PopD['Total consumption\n(GWh):\nDomestic\n']/Gas_Consumption_Cleaned_PopD['Population']
Gas_Consumption_Cleaned_PopD['Commercial_Gas_Consumption(GWh/person)'] = Gas_Consumption_Cleaned_PopD['Total consumption\n(GWh):\nNon-Domestic']/Gas_Consumption_Cleaned_PopD['Population']
Gas_Consumption_Cleaned_PopD['Total_Gas_Consumption(GWh/person)'] = Gas_Consumption_Cleaned_PopD['Total consumption\n(GWh):\nAll meters']/Gas_Consumption_Cleaned_PopD['Population']

#Please note only domestic values where used in visualisation as the scope of this project centres around the domestic cost of energy
Gas_Consumption_Cleaned_PopD

In [50]:
#Setting the maximum and minimum values as variables
Max_Domestic_Gas_Consumption = Gas_Consumption_Cleaned_PopD['Domestic_Gas_Consumption(GWh/person)'].max()
Min_Domestic_Gas_Consumption = Gas_Consumption_Cleaned_PopD['Domestic_Gas_Consumption(GWh/person)'].min()

In [51]:
#defining the graphing function
def create_domestic_choropleth(year):
    fig = px.choropleth(Gas_Consumption_Cleaned_PopD.loc[Gas_Consumption_Cleaned_PopD['Year'] == year]
                        ,locations='Location_Code'
                        ,geojson=Regions
                        ,color ='Domestic_Gas_Consumption(GWh/person)'
                        ,featureidkey="properties.NUTS112CD"
                        ,hover_name='Region'
                        ,range_color = (Min_Domestic_Gas_Consumption, Max_Domestic_Gas_Consumption))
    fig.update_geos(
        fitbounds="geojson",
       # projection_scale = 100,
        visible=False,
        projection_type="orthographic" 
        )
    fig.update_layout(
    margin={"r": 0, "t": 50, "l": 0, "b": 0}, 
    title={
        'text': f"{year}",  
        'y': 0.95, 
        'x': 0.5,  
        'xanchor': 'center',
        'yanchor': 'top'},
    coloraxis_colorbar_title=None,
    coloraxis_colorbar=dict(
            len=0.8,
            thickness=15 
        ),
        width=800,
        height=650,
        font=dict(
        size=26 
    ))

    return fig

#Showing "Domestic Gas Consumption in {year} (GWh/person)"


In [ ]:
#Defining the years to graph as a variable
Years_to_Graph = list(range(2005,2023))
Years_to_Graph

In [ ]:
#Looping the create a gragh function for each year
#Choropleth hidden here to support GitHub upload
figures = [create_domestic_choropleth(year) for year in Years_to_Graph]

for fig in figures:
    fig.show()

This is the Importation of Energy Bills data

In [54]:
#The importation of Energy price data

#Importing the Historical data sheet (Pre-2020 data)
Energy_Price_Raw_S1 = pd.read_excel(io=Domestic_Energy_Bills_5, sheet_name=r'2.2.3 (Historic Consumption)', header=16, usecols='A:F')
#Filtering to only include the average value for each region for each year
Energy_Price_Raw_S1 = Energy_Price_Raw_S1[Energy_Price_Raw_S1['Bill range\n[Note 1]'] == 'Average']
#Selecting only the needed columns
Energy_Price_Raw_S1 = Energy_Price_Raw_S1[['Year', 'PES area', 'Credit: Unit cost (Pence per kWh)']]
#Renaming columns to fit with the same naming convention as the recent dataset
Energy_Price_Raw_S1.rename(columns={'PES area':'Region','Credit: Unit cost (Pence per kWh)':'Average Unit Cost (Pence per kWh)'}, inplace = True)
#Removing Yearly totals. These will be calculated later using an aggregate function
Energy_Price_Raw_S1 = Energy_Price_Raw_S1[Energy_Price_Raw_S1['Region'] != 'United Kingdom']
#(!!)Removing years later than 2017 as these are included in both datasets. Include dialog around the difference in these years. The latest dataset was chosen under the assumption that the more recent data would be better. justify this
Energy_Price_Raw_S1 = Energy_Price_Raw_S1[Energy_Price_Raw_S1['Year'] < 2017]

#Importing the Historical data sheet (Post-2018 data)
Energy_Price_Raw_S2 = pd.read_excel(io=Domestic_Energy_Bills_5, sheet_name=r'2.2.3', header=11,usecols='A:C')
#Removing Yearly totals. These will be calculated later using an aggregate function
Energy_Price_Raw_S2 = Energy_Price_Raw_S2[Energy_Price_Raw_S2['Region'] != 'United Kingdom']
Energy_Price_Raw_S2.rename(columns={'Credit: Unit cost (Pence per kWh)':'Average Unit Cost (Pence per kWh)'}, inplace = True)

Energy_Price = pd.concat([Energy_Price_Raw_S1,Energy_Price_Raw_S2])


In [ ]:
Energy_Price[Energy_Price['Year'] == 2000]

In [56]:
#ChatGPT Generated code to replace 'South West' with the average of 'Southern' and 'South West'
# Group by Year to calculate averages
for year in Energy_Price['Year'].unique():
    # Filter the current year's data for 'Southern' and 'South West'
    southern_value = Energy_Price.loc[(Energy_Price['Year'] == year) & (Energy_Price['Region'] == 'Southern'), 'Average Unit Cost (Pence per kWh)'].values
    south_west_value = Energy_Price.loc[(Energy_Price['Year'] == year) & (Energy_Price['Region'] == 'South West'), 'Average Unit Cost (Pence per kWh)'].values

    if southern_value.size > 0 and south_west_value.size > 0:
        # Calculate the average
        avg_value = (southern_value[0] + south_west_value[0]) / 2

        # Update 'South West' value
        Energy_Price.loc[(Energy_Price['Year'] == year) & (Energy_Price['Region'] == 'South West'), 'Average Unit Cost (Pence per kWh)'] = avg_value

In [ ]:
#ChatGPT Generated code to replace 'South East' with the average of 'Southern' and 'South East'
# Group by Year to calculate averages
for year in Energy_Price['Year'].unique():
    # Filter the current year's data for 'Southern' and 'South East'
    southern_value = Energy_Price.loc[(Energy_Price['Year'] == year) & (Energy_Price['Region'] == 'Southern'), 'Average Unit Cost (Pence per kWh)'].values
    south_east_value = Energy_Price.loc[(Energy_Price['Year'] == year) & (Energy_Price['Region'] == 'South East'), 'Average Unit Cost (Pence per kWh)'].values

    if southern_value.size > 0 and south_east_value.size > 0:
        # Calculate the average
        avg_value = (southern_value[0] + south_east_value[0]) / 2

        # Update 'South East' value
        Energy_Price.loc[(Energy_Price['Year'] == year) & (Energy_Price['Region'] == 'South East'), 'Average Unit Cost (Pence per kWh)'] = avg_value

In [57]:
#This process above assumes that each region is of roughly the same magnitude. Interpolation was used

In [58]:
Energy_Price.rename(columns={'Region':'In_Region'}, inplace = True)

In [ ]:
#Standardisation of naming convention for Region
Energy_Price['In_Region'].unique()
Energy_Price_Region_Correction = pd.DataFrame({'In_Region':['West Midlands',
       'South East', 'Eastern',
       'Yorkshire', 'London', 'North West',
       'North East', 'East Midlands', 'South West'], 'Region':['West Midlands','South East','East','Yorkshire and The Humber','London', 'North West',
       'North East', 'East Midlands', 'South West']})
Energy_Price_Region_Correction

In [60]:
Energy_Price_Region = pd.merge(Energy_Price, Energy_Price_Region_Correction, on='In_Region', how='left')
Energy_Price_Region = Energy_Price_Region[Energy_Price_Region['Region'].notnull()]

In [ ]:
#Data processing to allow for the line graph of energy price by year
Energy_Price_Aggregate = Energy_Price.groupby('Year',as_index=False).agg({'Average Unit Cost (Pence per kWh)' : 'mean'})
Energy_Price_Aggregate

Now to bring in the Gas inmport cost data

In [ ]:
#Importation of the Price of Fuel Imports data
Energy_Generation_Imports_Raw = pd.read_excel(io=Energy_Import_Costs_6, sheet_name=r'3.2.1 (Annual real)', header=12, usecols='A:J')
Energy_Generation_Imports_Raw.rename(columns={'Major power producers: Coal (pence per kWh)\n[Note 1]':'Coal (pence per kWh)','Major power producers: Oil (pence per kWh)\n[Note 2, 3]':'Oil (pence per kWh)','Major power producers: Natural gas (pence per kWh)\n[Note 4]':'Natural Gas (pence per kWh)'}, inplace = True)
Energy_Generation_Imports_Subset = Energy_Generation_Imports_Raw[['Year','Coal (pence per kWh)', 'Oil (pence per kWh)','Natural Gas (pence per kWh)']]
Energy_Generation_Imports_Subset = Energy_Generation_Imports_Subset[Energy_Generation_Imports_Raw['Year'] >= 1998]

Energy_Generation_Imports_Subset

In [ ]:
Energy_Vs_Import_Price = Energy_Generation_Imports_Subset.merge(Energy_Price_Aggregate, on='Year')
#Energy_Vs_Import_Price['Log Unit Cost'] = np.log(Energy_Vs_Import_Price['Credit: Unit cost (Pence per kWh)'])
#Energy_Vs_Import_Price['Log Import Cost'] = np.log(Energy_Vs_Import_Price['Major power producers: Oil (pence per kWh)\n[Note 2, 3]'])
Energy_Vs_Import_Price

In [ ]:
#Median Income by region and region Adjusted for inflation
Median_Income_Raw = pd.read_excel(io=Average_Earning_7, sheet_name='FT weekly pay by home region', header=4, nrows=9, skiprows=[5,6,7], usecols='B:E,G:H, J:N, P:Y, AA:AC')
Median_Income_Raw.rename(columns={'Unnamed: 1': 'Region'}, inplace=True)
Median_Income_Raw

In [65]:
# Unpivot the Median_Income_Raw df
Median_Income = pd.melt(
    Median_Income_Raw, 
    id_vars=['Region'], 
    var_name='Year',   
    value_name='Median_Income' )

#Convert the weekly income into salary
Median_Income['Salary'] = Median_Income['Median_Income']*52

In [ ]:
Median_Income_by_Year = Median_Income.groupby('Year', as_index=False)['Salary'].mean()
Median_Income_by_Year

In [ ]:
Energy_Vs_Import_Price_Sal = Energy_Vs_Import_Price.merge(Median_Income_by_Year, on=['Year'], how='left')
Energy_Vs_Import_Price_Sal = Energy_Vs_Import_Price_Sal[Energy_Vs_Import_Price_Sal['Salary'].notnull()]
Energy_Vs_Import_Price_Sal = Energy_Vs_Import_Price_Sal.reset_index()
Energy_Vs_Import_Price_Sal = Energy_Vs_Import_Price_Sal.drop(columns=['index'])
Energy_Vs_Import_Price_Sal["Salary (£'000)"] = Energy_Vs_Import_Price_Sal['Salary']/1000
Energy_Vs_Import_Price_Sal

In [ ]:
Energy_Price_Vs_Salary_Fig = px.line(Energy_Vs_Import_Price_Sal
                                     , x='Year'
                                     , y=['Average Unit Cost (Pence per kWh)','Oil (pence per kWh)', 'Natural Gas (pence per kWh)', 'Coal (pence per kWh)' ]
                                     ,color_discrete_map= { 
                                                             
                                                                
                                                                'Average Unit Cost (Pence per kWh)': 'green',
                                                                'Oil (pence per kWh)': 'grey',
                                                                'Coal (pence per kWh)': 'lightgrey',
                                                                'Natural Gas (pence per kWh)': 'purple'})
Energy_Price_Vs_Salary_Fig.update_layout(
        margin={"r": 10, "t": 130, "l": 10, "b": 10},
    title="Unit Cost of Energy and Import Cost of Fuel (2002-2023)",
    xaxis_title="Year",
    yaxis_title="",
    title_x=0.5,
    title_y=0.95, 
    legend=dict(
        orientation="h", 
        yanchor="bottom", 
        y=1, 
        xanchor="center",  
        x=0.5,
        title = None
    ,font=dict(
        size=26 
    )),
    height = 1000,
    font=dict(
        size=26 
    ))
#Energy_Vs_Import_Price_Fig = px.line(Energy_Vs_Import_Price, x='Year', y=['Oil (pence per kWh)','Average Unit Cost (Pence per kWh)'])
Energy_Price_Vs_Salary_Fig

In [ ]:
Energy_Price_Vs_Salary_Fig = px.line(Energy_Vs_Import_Price_Sal
                                     , x='Year'
                                     , y=["Salary (£'000)",'Average Unit Cost (Pence per kWh)','Oil (pence per kWh)', 'Natural Gas (pence per kWh)', 'Coal (pence per kWh)' ]
                                     ,color_discrete_map= { 
                                                                "Salary (£'000)":'Orange',
                                                                'Average Unit Cost (Pence per kWh)': 'green',
                                                                'Oil (pence per kWh)': 'grey',
                                                                'Coal (pence per kWh)': 'lightgrey',
                                                                'Natural Gas (pence per kWh)': 'purple'})
Energy_Price_Vs_Salary_Fig.update_layout(
        margin={"r": 10, "t": 170, "l": 10, "b": 10},
    title="Salary Against Unit Cost of Energy and Import Cost of Fuel (2002-2023)",
    xaxis_title="Year",
    yaxis_title="",
    title_x=0.5,
    title_y=0.95, 
    legend=dict(
        orientation="h", 
        yanchor="bottom", 
        y=1, 
        xanchor="center",  
        x=0.5,
        title = None
    ,font=dict(
        size=26 
    )),
    height = 1000,
    font=dict(
        size=26 
    ))
#Energy_Vs_Import_Price_Fig = px.line(Energy_Vs_Import_Price, x='Year', y=['Oil (pence per kWh)','Average Unit Cost (Pence per kWh)'])
Energy_Price_Vs_Salary_Fig

Up next do corrolation matrixs on raw data
do a time series corrolation on the graph values


In [ ]:
#Energy_Generation_Imports_Subset_Matrix = Energy_Generation_Imports_Subset.corr()
#plt.imshow(Energy_Generation_Imports_Subset_Matrix, cmap = 'Blues')
Energy_Vs_Import_Price_Matrix = Energy_Vs_Import_Price.corr()

sns.heatmap(Energy_Vs_Import_Price_Matrix, cmap="Reds", annot=True)

plt.title("Correlation Between Import Price of Fuel and Average Unit Cost of Energy (1998-2023)", pad=20)

plt.show()


In [ ]:
Years_to_Graph = list(range(2005,2023))
Years_to_Graph

In [ ]:
Gas_Consumption_Cleaned_PopD_MedIncome = Gas_Consumption_Cleaned_PopD.merge(Median_Income, on=['Region','Year'], how='left')
Gas_Consumption_Cleaned_PopD_MedIncome['Domestic_Gas_Consumption_to_Median_Salary(GWh/person/£)'] = Gas_Consumption_Cleaned_PopD_MedIncome['Domestic_Gas_Consumption(GWh/person)']/Gas_Consumption_Cleaned_PopD_MedIncome['Salary']
Gas_Consumption_Cleaned_PopD_MedIncome

In [ ]:
Energy_Vs_Import_Price_Sal = Energy_Vs_Import_Price.merge(Median_Income, on=['Year'], how='left')
Energy_Vs_Import_Price_Sal

EDITS MADE HERE

In [ ]:
#retrieving the yearly minimum of 'Domestic_Gas_Consumption_to_Median_Salary(GWh/person/£)'
Gas_Consumption_Cleaned_PopD_MedIncome_Min_Values = Gas_Consumption_Cleaned_PopD_MedIncome.groupby('Year',as_index=False).agg({
    'Domestic_Gas_Consumption_to_Median_Salary(GWh/person/£)' : 'min'
})
Gas_Consumption_Cleaned_PopD_MedIncome_Min_Values.rename(columns= {'Domestic_Gas_Consumption_to_Median_Salary(GWh/person/£)': 'Domestic_Gas_Consumption_to_Median_Salary_Annual_min(GWh/person/£)'},inplace=True)
Gas_Consumption_Cleaned_PopD_MedIncome_Min_Values

In [ ]:
#Joining the Yearly minimum onto the main table and expressing Domestic_Gas_Consumption_to_Median_Salary(GWh/person/£) as a percentage of the annual minimum
Gas_Consumption_Cleaned_PopD_MedIncome_Percent = Gas_Consumption_Cleaned_PopD_MedIncome.merge(Gas_Consumption_Cleaned_PopD_MedIncome_Min_Values, on=['Year'], how='left')
Gas_Consumption_Cleaned_PopD_MedIncome_Percent['Domestic_Gas_Consumption_to_Median_Salary_as_a_Percentage_of_Annual_min'] = Gas_Consumption_Cleaned_PopD_MedIncome_Percent['Domestic_Gas_Consumption_to_Median_Salary(GWh/person/£)']/Gas_Consumption_Cleaned_PopD_MedIncome_Percent['Domestic_Gas_Consumption_to_Median_Salary_Annual_min(GWh/person/£)']
Gas_Consumption_Cleaned_PopD_MedIncome_Percent

In [ ]:
#Choropleth standardised to see if north and south are getting more or less equal
#Choropleth hidden here to support GitHub upload
def create_salaryration_choropleth(year):
    fig = px.choropleth(Gas_Consumption_Cleaned_PopD_MedIncome.loc[Gas_Consumption_Cleaned_PopD_MedIncome['Year'] == year]
                        ,locations='Location_Code'
                        ,geojson=Regions
                        ,color ='Domestic_Gas_Consumption_to_Median_Salary(GWh/person/£)'
                        ,featureidkey="properties.NUTS112CD"
                        ,hover_name='Region'
                        ,color_continuous_scale=px.colors.sequential.Viridis
                        )
    fig.update_geos(
        fitbounds="geojson",
       # projection_scale = 100,
        visible=False,
        projection_type="orthographic" 
        )
    fig.update_layout(
    margin={"r": 0, "t": 50, "l": 0, "b": 0}, 
    title={
        'text': f"{year}",  
        'y': 0.95, 
        'x': 0.5,  
        'xanchor': 'center',
        'yanchor': 'top'},
    coloraxis_colorbar_title=None,
    coloraxis_colorbar=dict(
            len=0.8,
            thickness=15 
        ),
        width=800,
        height=650,
        font=dict(
        size=26 
    ))

    return fig

Years_to_Graph = list(range(2005,2023))
Years_to_Graph

figures = [create_salaryration_choropleth(year) for year in Years_to_Graph]

for fig in figures:
    fig.show()
    
    #Showing "Domestic Gas Consumption as a Portion of Salary {year} (GWh/person/£)"

In [104]:
#defining the max and min values for all years
Max_Percent = Gas_Consumption_Cleaned_PopD_MedIncome_Percent['Domestic_Gas_Consumption_to_Median_Salary_as_a_Percentage_of_Annual_min'].max()
Min_Percent = Gas_Consumption_Cleaned_PopD_MedIncome_Percent['Domestic_Gas_Consumption_to_Median_Salary_as_a_Percentage_of_Annual_min'].min()

In [ ]:
#Choropleth standardised to see if north and south are getting more or less equal
#Choropleth hidden here to support GitHub upload
def create_salaryration_choropleth(year):
    fig = px.choropleth(Gas_Consumption_Cleaned_PopD_MedIncome_Percent.loc[Gas_Consumption_Cleaned_PopD_MedIncome_Percent['Year'] == year]
                        ,locations='Location_Code'
                        ,geojson=Regions
                        ,color ='Domestic_Gas_Consumption_to_Median_Salary_as_a_Percentage_of_Annual_min'
                        ,featureidkey="properties.NUTS112CD"
                        ,hover_name='Region'
                        ,color_continuous_scale=px.colors.sequential.Viridis
                        ,range_color = (Min_Percent, Max_Percent)
                        )
    fig.update_geos(
        fitbounds="geojson",
       # projection_scale = 100,
        visible=False,
        projection_type="orthographic" 
        )
    fig.update_layout(
    margin={"r": 0, "t": 50, "l": 0, "b": 0}, 
    title={
        'text': f"{year}",  
        'y': 0.95, 
        'x': 0.5,  
        'xanchor': 'center',
        'yanchor': 'top'},
    coloraxis_colorbar_title=None,
    coloraxis_colorbar=dict(
            len=0.8,
            thickness=15 
        ),
        width=800,
        height=650,
        font=dict(
        size=26 
    ))

    return fig

Years_to_Graph = list(range(2005,2023))
Years_to_Graph

figures = [create_salaryration_choropleth(year) for year in Years_to_Graph]

for fig in figures:
    fig.show()
    
    #Showing "Domestic Gas Consumption as a Portion of Salary {year} % difference From Annual Minimum (%)"

In [ ]:
Gas_Consumption_Cleaned_PopD_MedIncome

In [107]:
#Join to allow generation of choropleths displaying Energy cost againt salary by region and year
Gas_Consumption_Cleaned_PopD_MedIncome_EngPrice = pd.merge(Gas_Consumption_Cleaned_PopD_MedIncome, Energy_Price_Region, on=['Region','Year'], how='left')
Gas_Consumption_Cleaned_PopD_MedIncome_EngPrice["Salary (£'000)"] = Gas_Consumption_Cleaned_PopD_MedIncome_EngPrice["Salary"]/1000
Gas_Consumption_Cleaned_PopD_MedIncome_EngPrice["EngPrice / Salary (Pence/kWh/£'000)"] = Gas_Consumption_Cleaned_PopD_MedIncome_EngPrice['Average Unit Cost (Pence per kWh)']/Gas_Consumption_Cleaned_PopD_MedIncome_EngPrice["Salary (£'000)"]

In [108]:
Max_Unit_Cost = Gas_Consumption_Cleaned_PopD_MedIncome_EngPrice["Average Unit Cost (Pence per kWh)"].max()
Min_Unit_Cost = Gas_Consumption_Cleaned_PopD_MedIncome_EngPrice["Average Unit Cost (Pence per kWh)"].min()

In [ ]:
#Choropleth showing avg energy price agianst alary by region and year (This one for comparing change over time)
#Choropleth hidden here to support GitHub upload
def create_salary_to_engprice_choropleth_time(year):
    fig = px.choropleth(Gas_Consumption_Cleaned_PopD_MedIncome_EngPrice.loc[Gas_Consumption_Cleaned_PopD_MedIncome_EngPrice['Year'] == year]
                        ,locations='Location_Code'
                        ,geojson=Regions
                        ,color ="Average Unit Cost (Pence per kWh)"
                        ,featureidkey="properties.NUTS112CD"
                        ,hover_name='Region'
                        ,range_color = (Min_Unit_Cost, Max_Unit_Cost)
                        ,color_continuous_scale = px.colors.diverging.RdYlBu[::-1]
                        )
    fig.update_geos(
        fitbounds="geojson",
       # projection_scale = 100,
        visible=False,
        projection_type="orthographic" 
        )
    fig.update_layout(
    margin={"r": 0, "t": 50, "l": 0, "b": 0}, 
    title={
        'text': f"{year}",  
        'y': 0.95, 
        'x': 0.5,  
        'xanchor': 'center',
        'yanchor': 'top'},
    coloraxis_colorbar_title=None,
    coloraxis_colorbar=dict(
            len=0.8,
            thickness=15 
        ),
        width=800,
        height=650,
        font=dict(
        size=26 
    ))

    return fig

Years_to_Graph = list(range(2005,2023))
Years_to_Graph

figures = [create_salary_to_engprice_choropleth_time(year) for year in Years_to_Graph]

for fig in figures:
    fig.show()
    
    #Showing "Average Energy Price {year} (Pence/kWh)"
    
    ##Do not include

In [110]:
Max_Price_Sal_Ratio = Gas_Consumption_Cleaned_PopD_MedIncome_EngPrice["EngPrice / Salary (Pence/kWh/£'000)"].max()
Min_Price_Sal_Ratio = Gas_Consumption_Cleaned_PopD_MedIncome_EngPrice["EngPrice / Salary (Pence/kWh/£'000)"].min()

In [ ]:
Gas_Consumption_Cleaned_PopD_MedIncome_EngPrice[Gas_Consumption_Cleaned_PopD_MedIncome_EngPrice['Year']==2022]

In [ ]:
#retrieving the yearly minimum of "EngPrice / Salary (Pence/kWh/£'000)"
Gas_Consumption_Cleaned_PopD_MedIncome_EngPrice_Min_Values = Gas_Consumption_Cleaned_PopD_MedIncome_EngPrice.groupby('Year',as_index=False).agg({
    "EngPrice / Salary (Pence/kWh/£'000)" : 'min'
})
Gas_Consumption_Cleaned_PopD_MedIncome_EngPrice_Min_Values.rename(columns= {"EngPrice / Salary (Pence/kWh/£'000)": "EngPrice / Salary of Annual Minimum(Pence/kWh/£'000)"},inplace=True)
Gas_Consumption_Cleaned_PopD_MedIncome_EngPrice_Min_Values

In [ ]:
#Joining the Yearly minimum onto the main table and expressing "EngPrice / Salary (Pence/kWh/£'000)" as a percentage of the annual minimum
Gas_Consumption_Cleaned_PopD_MedIncome_EngPrice_Percent = Gas_Consumption_Cleaned_PopD_MedIncome_EngPrice.merge(Gas_Consumption_Cleaned_PopD_MedIncome_EngPrice_Min_Values, on=['Year'], how='left')
Gas_Consumption_Cleaned_PopD_MedIncome_EngPrice_Percent["EngPrice / Salary as a Percent of Annual Minimum(%)"] = Gas_Consumption_Cleaned_PopD_MedIncome_EngPrice_Percent["EngPrice / Salary (Pence/kWh/£'000)"]/Gas_Consumption_Cleaned_PopD_MedIncome_EngPrice_Percent["EngPrice / Salary of Annual Minimum(Pence/kWh/£'000)"]
Gas_Consumption_Cleaned_PopD_MedIncome_EngPrice_Percent

In [114]:
Max_EngPrice_Sal_Perc = Gas_Consumption_Cleaned_PopD_MedIncome_EngPrice_Percent["EngPrice / Salary as a Percent of Annual Minimum(%)"].max()
Min_EngPrice_Sal_Perc = Gas_Consumption_Cleaned_PopD_MedIncome_EngPrice_Percent["EngPrice / Salary as a Percent of Annual Minimum(%)"].min()

In [ ]:

#Choropleth hidden here to support GitHub upload
def create_salary_to_engprice_choropleth_time(year):
    fig = px.choropleth(Gas_Consumption_Cleaned_PopD_MedIncome_EngPrice_Percent.loc[Gas_Consumption_Cleaned_PopD_MedIncome_EngPrice_Percent['Year'] == year]
                        ,locations='Location_Code'
                        ,geojson=Regions
                        ,color ="EngPrice / Salary as a Percent of Annual Minimum(%)"
                        ,featureidkey="properties.NUTS112CD"
                        ,hover_name='Region'
                        ,range_color = (Min_EngPrice_Sal_Perc, Max_EngPrice_Sal_Perc)
                        ,color_continuous_scale = px.colors.diverging.RdYlBu[::-1]
                        )
    fig.update_geos(
        fitbounds="geojson",
       # projection_scale = 100,
        visible=False,
        projection_type="orthographic" 
        )
    fig.update_layout(
    margin={"r": 0, "t": 50, "l": 0, "b": 0}, 
    title={
        'text': f"{year}",  
        'y': 0.95, 
        'x': 0.5,  
        'xanchor': 'center',
        'yanchor': 'top'},
    coloraxis_colorbar_title=None,
    coloraxis_colorbar=dict(
            len=0.8,
            thickness=15 
        ),
        width=800,
        height=650,
        font=dict(
        size=26 
    ))

    return fig

Years_to_Graph = list(range(2005,2023))
Years_to_Graph

figures = [create_salary_to_engprice_choropleth_time(year) for year in Years_to_Graph]

for fig in figures:
    fig.show()
    
    #showing "Average Energy Price as a Portion of Salary as a Percentage of the annual minimum (%)"

In [ ]:
#Choropleth showing avg energy price against salary by region and year (This one for comparing equality of north vs south)
#Choropleth hidden here to support GitHub upload
def create_salary_to_engprice_choropleth(year):
    fig = px.choropleth(Gas_Consumption_Cleaned_PopD_MedIncome_EngPrice.loc[Gas_Consumption_Cleaned_PopD_MedIncome_EngPrice['Year'] == year]
                        ,locations='Location_Code'
                        ,geojson=Regions
                        ,color ="EngPrice / Salary (Pence/kWh/£'000)"
                        ,featureidkey="properties.NUTS112CD"
                        ,hover_name='Region'
                        ,color_continuous_scale=px.colors.diverging.RdYlBu[::-1]
                        )
    fig.update_geos(
        fitbounds="geojson",
       # projection_scale = 100,
        visible=False,
        projection_type="orthographic" 
        )
    fig.update_layout(
    margin={"r": 0, "t": 50, "l": 0, "b": 0}, 
    title={
        'text': f"{year}",  
        'y': 0.95, 
        'x': 0.5,  
        'xanchor': 'center',
        'yanchor': 'top'},
    coloraxis_colorbar_title=None,
    coloraxis_colorbar=dict(
            len=0.8,
            thickness=15 
        ),
        width=800,
        height=650,
        font=dict(
        size=26 
    ))
    
    return fig

Years_to_Graph = list(range(2005,2023))
Years_to_Graph

figures = [create_salary_to_engprice_choropleth(year) for year in Years_to_Graph]

for fig in figures:
    fig.show()
    
    #Showsing "Average Energy Price as a Portion of Salary {year} (Pence/kWh/£'000)" (Standardised)

In [ ]:
#Choropleth showing avg energy price agianst alary by region and year (This one for comparing change over time)
#Choropleth hidden here to support GitHub upload
def create_salary_to_engprice_choropleth_time(year):
    fig = px.choropleth(Gas_Consumption_Cleaned_PopD_MedIncome_EngPrice.loc[Gas_Consumption_Cleaned_PopD_MedIncome_EngPrice['Year'] == year]
                        ,locations='Location_Code'
                        ,geojson=Regions
                        ,color ="EngPrice / Salary (Pence/kWh/£'000)"
                        ,featureidkey="properties.NUTS112CD"
                        ,hover_name='Region'
                        ,range_color = (Min_Price_Sal_Ratio, Max_Price_Sal_Ratio)
                        ,color_continuous_scale = px.colors.diverging.RdYlBu[::-1]
                        )
    fig.update_geos(
        fitbounds="geojson",
       # projection_scale = 100,
        visible=False,
        projection_type="orthographic" 
        )
    fig.update_layout(
    margin={"r": 0, "t": 50, "l": 0, "b": 0}, 
    title={
        'text': f"{year}",  
        'y': 0.95, 
        'x': 0.5,  
        'xanchor': 'center',
        'yanchor': 'top'},
    coloraxis_colorbar_title=None,
    coloraxis_colorbar=dict(
            len=0.8,
            thickness=15 
        ),
        width=800,
        height=650,
        font=dict(
        size=26 
    ))

    return fig

Years_to_Graph = list(range(2005,2023))
Years_to_Graph

figures = [create_salary_to_engprice_choropleth_time(year) for year in Years_to_Graph]

for fig in figures:
    fig.show()
    
    #showing "Average Energy Price as a Portion of Salary {year} (Pence/kWh/£'000)" (on a fixed scale)